### Imports

In [ ]:
import logging

from IPython.core.display import display, HTML

from scripts.main import *
from scripts.retrieve_data import *
from scripts.extract_data import*

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# set the width of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))

###  Run the whole pipeline

In [ ]:
run()

In [ ]:
logging.shutdown()

### Load the configuration

In [ ]:
config = load_config()

### Get the series

In [ ]:
df = extract_transform_and_save_data_from_files(config)

### Do the grouping by

In [ ]:
df, df_count_series, df_count_studies = do_series_groupby(config, df)
display(df_count_series)
display(df_count_studies)
display(df_count_series.groupby('Machine Group').sum())
display(df_count_studies.groupby('Machine Group').sum())

### Clean up the data (while changing things in the API so the bugs/errors do not appear anymore)

In [ ]:
df = df[~df['Study Instance UID'].isin(df[df['Institution Name'] == 'Hopital neuchatelois']['Study Instance UID'])]
df = df[~df['Study Instance UID'].isin(df[df['Institution Name'] == 'MEDECINE NUCLEAIRE']['Study Instance UID'])]
df = df[~df['Study Instance UID'].isin(df[df['Machine'] == '']['Study Instance UID'])]

### Check why some non-valid institution name went through the filter

In [ ]:
df_bad_series = df[df['Study Instance UID'].isin(df[df['Machine'] == 'Ingenuity TF PET/CT']['Study Instance UID'])]

if len(df_bad_series) > 0:
    bad_study_UID = list(set(df_bad_series['Study Instance UID'].values))[0]
    logging.info('bad_study_UID: ' + bad_study_UID)
    
    inst_name = list(set([inst_name.replace('  ', ' ') for inst_name in df_bad_series.loc[:, 'Institution Name']]))[0]
    logging.info('inst_name: ' + inst_name)
    logging.info('accepted_inst_names: ' + str(accepted_inst_names))
    logging.info('inst_name is in accepted_inst_names? ' + str(inst_name.lower().replace(' ', '') in accepted_inst_names))
    logging.info('date: ' + str(list(set(df_bad_series['Series Date']))[0]))
    
    df_studies = find_studies_for_day(config, '20191029', ['PT', 'NM'])
    df_bad_study = df_studies[df_studies['Study Instance UID'] == bad_study_UID]
    df_series = find_series_for_studies(config, df_bad_study)
    df_series

### Solve the problem of having some studies with mixed machine names

In [ ]:
# find a study which has both the millennium and another machine
df_series_for_study = df[df['Study Instance UID'].isin(df[df['Machine'] == 'MILLENNIUM MPR']['Study Instance UID'])]
df_series_for_study[df_series_for_study['Study Instance UID'].isin(df_series_for_study[df_series_for_study['Machine'] == 'BrightSpeed']['Study Instance UID'])]

### Debug the processing of some unfetchable rows

### Close the logging

In [ ]:
logging.shutdown()